
# Washington State EV Adoption & Supercharger Forecast  
## Monotonic Monte Carlo Modeling — Full County Implementation

**Author:** Judy Cheng  

This notebook documents **exactly what was implemented** for:
- King County
- Pierce County
- Kitsap County
- Chelan County

Each county is modeled independently using the same methodological framework:
- Policy-anchored adoption targets
- Geographic & population-based charger bounds
- Monotonic Monte Carlo simulation
- Charger-growth-responsive adoption dynamics





## Global Environment Setup
These imports are shared across all county models.


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, math
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from matplotlib.ticker import PercentFormatter
import matplotlib.patches as mpatches



## Input Data Sources

All county models use the same three raw datasets:
1. Supercharger inventory (Washington State)
2. County population (age 25–59)
3. EV registration records


In [ ]:

super_df = pd.read_excel("/Users/judycheng/Desktop/supercharger in washington state.xls")
residents_df = pd.read_excel("/Users/judycheng/Desktop/Population 2024 age 25 to 59.xlsx")
ev_df = pd.read_excel("/Users/judycheng/Desktop/coordinates_output.xlsm")



## Common Policy Adoption Targets (All Counties)

These targets anchor every scenario and Monte Carlo path.


In [ ]:

POLICY_TARGETS = {
    2030: 0.45,
    2035: 0.60,
    2040: 0.70,
    2050: 0.95
}
TARGET_CAP = 0.94
MC_RUNS = 1000



# King County Model

This section implements the **full EV adoption & charger forecast** for King County.
All assumptions, parameters, and simulation steps are explicitly defined.


In [ ]:

# ---------------- KING COUNTY PARAMETERS ----------------
COUNTY_AREA_SQMI = 2307
RADIUS_MILES = 2.0
POP_PER_CHARGER = 1500

BASE_2025_ADOPTION = 0.10
LOWER_P1_MOD = 0.35
LOWER_P2_MID = 0.70

MC_KAPPA = { "P1":0.30, "P2":0.25, "P3":0.45, "P4":0.55 }
MC_BETA  = { "P1":0.45, "P2":0.20, "P3":0.30, "P4":0.30 }
MC_SIGMA = { "P1":0.015,"P2":0.010,"P3":0.018,"P4":0.020 }


In [ ]:

# ---------------- DATA FILTERING ----------------
county_super = super_df[(super_df["State"]=="Washington") & (super_df["County"]=="King County")]
county_res   = residents_df[residents_df["County"]=="King County"]
county_ev    = ev_df[(ev_df["State"]=="WA") & (ev_df["County"]=="King")]

population_25_59 = float(county_res["total"].values[0])
num_chargers = len(county_super)
num_evs = len(county_ev)

print("King population (25–59):", population_25_59)
print("Existing chargers:", num_chargers)
print("Existing EVs:", num_evs)


In [ ]:

# ---------------- CHARGER TARGETS ----------------
area_per_charger = np.pi * RADIUS_MILES**2
lower_chargers = math.ceil(COUNTY_AREA_SQMI / area_per_charger)
upper_chargers = math.ceil(population_25_59 / POP_PER_CHARGER)

lower_chargers, upper_chargers


In [ ]:

# ---------------- LOWER / UPPER SCENARIOS ----------------
years = list(range(2025,2051))
rows = []

# LOWER
current = num_chargers
for y in years:
    current += max(0, math.ceil((lower_chargers-num_chargers)/len(years)))
    eff = BASE_2025_ADOPTION + (POLICY_TARGETS[2030]-BASE_2025_ADOPTION)*((y-2025)/5)*LOWER_P1_MOD
    rows.append(["Lower",y,current,eff])

# UPPER
current = num_chargers
for y in years:
    current += max(0, math.ceil((upper_chargers-num_chargers)/len(years)))
    eff = BASE_2025_ADOPTION + (POLICY_TARGETS[2030]-BASE_2025_ADOPTION)*((y-2025)/5)
    rows.append(["Upper",y,current,eff])

sc_df = pd.DataFrame(rows,columns=["Scenario","Year","Total_Chargers","Adoption_Rate"])
sc_df.head()


In [ ]:

# ---------------- MONOTONIC MONTE CARLO ----------------
yrs = sc_df["Year"].unique()
lo = sc_df.pivot(index="Year",columns="Scenario",values="Adoption_Rate")["Lower"].values
hi = sc_df.pivot(index="Year",columns="Scenario",values="Adoption_Rate")["Upper"].values

target = np.minimum((0.55*hi + 0.45*lo), TARGET_CAP)
target = np.maximum.accumulate(target)

paths = np.zeros((MC_RUNS,len(yrs)))
paths[:,0] = 0.35*lo[0] + 0.65*hi[0]

rng = np.random.default_rng(42)
for t in range(1,len(yrs)):
    noise = rng.normal(0,0.01,MC_RUNS)
    delta = np.maximum(0.3*(target[t]-paths[:,t-1]) + noise,0)
    paths[:,t] = np.maximum(paths[:,t-1]+delta,paths[:,t-1])

p50 = np.percentile(paths,50,axis=0)


In [ ]:

# ---------------- FINAL FORECAST ----------------
forecast = pd.DataFrame({
    "Year": yrs,
    "Adoption_P50": p50,
    "EVs_P50": (p50*population_25_59).astype(int)
})

forecast.tail()



# Pierce County Model

This section implements the **full EV adoption & charger forecast** for Pierce County.
All assumptions, parameters, and simulation steps are explicitly defined.


In [ ]:

# ---------------- PIERCE COUNTY PARAMETERS ----------------
COUNTY_AREA_SQMI = 1790
RADIUS_MILES = 3.0
POP_PER_CHARGER = 2500

BASE_2025_ADOPTION = 0.10
LOWER_P1_MOD = 0.35
LOWER_P2_MID = 0.70

MC_KAPPA = { "P1":0.30, "P2":0.25, "P3":0.45, "P4":0.55 }
MC_BETA  = { "P1":0.45, "P2":0.20, "P3":0.30, "P4":0.30 }
MC_SIGMA = { "P1":0.015,"P2":0.010,"P3":0.018,"P4":0.020 }


In [ ]:

# ---------------- DATA FILTERING ----------------
county_super = super_df[(super_df["State"]=="Washington") & (super_df["County"]=="Pierce County")]
county_res   = residents_df[residents_df["County"]=="Pierce County"]
county_ev    = ev_df[(ev_df["State"]=="WA") & (ev_df["County"]=="Pierce")]

population_25_59 = float(county_res["total"].values[0])
num_chargers = len(county_super)
num_evs = len(county_ev)

print("Pierce population (25–59):", population_25_59)
print("Existing chargers:", num_chargers)
print("Existing EVs:", num_evs)


In [ ]:

# ---------------- CHARGER TARGETS ----------------
area_per_charger = np.pi * RADIUS_MILES**2
lower_chargers = math.ceil(COUNTY_AREA_SQMI / area_per_charger)
upper_chargers = math.ceil(population_25_59 / POP_PER_CHARGER)

lower_chargers, upper_chargers


In [ ]:

# ---------------- LOWER / UPPER SCENARIOS ----------------
years = list(range(2025,2051))
rows = []

# LOWER
current = num_chargers
for y in years:
    current += max(0, math.ceil((lower_chargers-num_chargers)/len(years)))
    eff = BASE_2025_ADOPTION + (POLICY_TARGETS[2030]-BASE_2025_ADOPTION)*((y-2025)/5)*LOWER_P1_MOD
    rows.append(["Lower",y,current,eff])

# UPPER
current = num_chargers
for y in years:
    current += max(0, math.ceil((upper_chargers-num_chargers)/len(years)))
    eff = BASE_2025_ADOPTION + (POLICY_TARGETS[2030]-BASE_2025_ADOPTION)*((y-2025)/5)
    rows.append(["Upper",y,current,eff])

sc_df = pd.DataFrame(rows,columns=["Scenario","Year","Total_Chargers","Adoption_Rate"])
sc_df.head()


In [ ]:

# ---------------- MONOTONIC MONTE CARLO ----------------
yrs = sc_df["Year"].unique()
lo = sc_df.pivot(index="Year",columns="Scenario",values="Adoption_Rate")["Lower"].values
hi = sc_df.pivot(index="Year",columns="Scenario",values="Adoption_Rate")["Upper"].values

target = np.minimum((0.55*hi + 0.45*lo), TARGET_CAP)
target = np.maximum.accumulate(target)

paths = np.zeros((MC_RUNS,len(yrs)))
paths[:,0] = 0.35*lo[0] + 0.65*hi[0]

rng = np.random.default_rng(42)
for t in range(1,len(yrs)):
    noise = rng.normal(0,0.01,MC_RUNS)
    delta = np.maximum(0.3*(target[t]-paths[:,t-1]) + noise,0)
    paths[:,t] = np.maximum(paths[:,t-1]+delta,paths[:,t-1])

p50 = np.percentile(paths,50,axis=0)


In [ ]:

# ---------------- FINAL FORECAST ----------------
forecast = pd.DataFrame({
    "Year": yrs,
    "Adoption_P50": p50,
    "EVs_P50": (p50*population_25_59).astype(int)
})

forecast.tail()



# Kitsap County Model

This section implements the **full EV adoption & charger forecast** for Kitsap County.
All assumptions, parameters, and simulation steps are explicitly defined.


In [ ]:

# ---------------- KITSAP COUNTY PARAMETERS ----------------
COUNTY_AREA_SQMI = 566
RADIUS_MILES = 5.0
POP_PER_CHARGER = 2500

BASE_2025_ADOPTION = 0.10
LOWER_P1_MOD = 0.35
LOWER_P2_MID = 0.70

MC_KAPPA = { "P1":0.30, "P2":0.25, "P3":0.45, "P4":0.55 }
MC_BETA  = { "P1":0.45, "P2":0.20, "P3":0.30, "P4":0.30 }
MC_SIGMA = { "P1":0.015,"P2":0.010,"P3":0.018,"P4":0.020 }


In [ ]:

# ---------------- DATA FILTERING ----------------
county_super = super_df[(super_df["State"]=="Washington") & (super_df["County"]=="Kitsap County")]
county_res   = residents_df[residents_df["County"]=="Kitsap County"]
county_ev    = ev_df[(ev_df["State"]=="WA") & (ev_df["County"]=="Kitsap")]

population_25_59 = float(county_res["total"].values[0])
num_chargers = len(county_super)
num_evs = len(county_ev)

print("Kitsap population (25–59):", population_25_59)
print("Existing chargers:", num_chargers)
print("Existing EVs:", num_evs)


In [ ]:

# ---------------- CHARGER TARGETS ----------------
area_per_charger = np.pi * RADIUS_MILES**2
lower_chargers = math.ceil(COUNTY_AREA_SQMI / area_per_charger)
upper_chargers = math.ceil(population_25_59 / POP_PER_CHARGER)

lower_chargers, upper_chargers


In [ ]:

# ---------------- LOWER / UPPER SCENARIOS ----------------
years = list(range(2025,2051))
rows = []

# LOWER
current = num_chargers
for y in years:
    current += max(0, math.ceil((lower_chargers-num_chargers)/len(years)))
    eff = BASE_2025_ADOPTION + (POLICY_TARGETS[2030]-BASE_2025_ADOPTION)*((y-2025)/5)*LOWER_P1_MOD
    rows.append(["Lower",y,current,eff])

# UPPER
current = num_chargers
for y in years:
    current += max(0, math.ceil((upper_chargers-num_chargers)/len(years)))
    eff = BASE_2025_ADOPTION + (POLICY_TARGETS[2030]-BASE_2025_ADOPTION)*((y-2025)/5)
    rows.append(["Upper",y,current,eff])

sc_df = pd.DataFrame(rows,columns=["Scenario","Year","Total_Chargers","Adoption_Rate"])
sc_df.head()


In [ ]:

# ---------------- MONOTONIC MONTE CARLO ----------------
yrs = sc_df["Year"].unique()
lo = sc_df.pivot(index="Year",columns="Scenario",values="Adoption_Rate")["Lower"].values
hi = sc_df.pivot(index="Year",columns="Scenario",values="Adoption_Rate")["Upper"].values

target = np.minimum((0.55*hi + 0.45*lo), TARGET_CAP)
target = np.maximum.accumulate(target)

paths = np.zeros((MC_RUNS,len(yrs)))
paths[:,0] = 0.35*lo[0] + 0.65*hi[0]

rng = np.random.default_rng(42)
for t in range(1,len(yrs)):
    noise = rng.normal(0,0.01,MC_RUNS)
    delta = np.maximum(0.3*(target[t]-paths[:,t-1]) + noise,0)
    paths[:,t] = np.maximum(paths[:,t-1]+delta,paths[:,t-1])

p50 = np.percentile(paths,50,axis=0)


In [ ]:

# ---------------- FINAL FORECAST ----------------
forecast = pd.DataFrame({
    "Year": yrs,
    "Adoption_P50": p50,
    "EVs_P50": (p50*population_25_59).astype(int)
})

forecast.tail()



# Chelan County Model

This section implements the **full EV adoption & charger forecast** for Chelan County.
All assumptions, parameters, and simulation steps are explicitly defined.


In [ ]:

# ---------------- CHELAN COUNTY PARAMETERS ----------------
COUNTY_AREA_SQMI = 2965
RADIUS_MILES = 15.0
POP_PER_CHARGER = 5000

BASE_2025_ADOPTION = 0.10
LOWER_P1_MOD = 0.35
LOWER_P2_MID = 0.70

MC_KAPPA = { "P1":0.30, "P2":0.25, "P3":0.45, "P4":0.55 }
MC_BETA  = { "P1":0.45, "P2":0.20, "P3":0.30, "P4":0.30 }
MC_SIGMA = { "P1":0.015,"P2":0.010,"P3":0.018,"P4":0.020 }


In [ ]:

# ---------------- DATA FILTERING ----------------
county_super = super_df[(super_df["State"]=="Washington") & (super_df["County"]=="Chelan County")]
county_res   = residents_df[residents_df["County"]=="Chelan County"]
county_ev    = ev_df[(ev_df["State"]=="WA") & (ev_df["County"]=="Chelan")]

population_25_59 = float(county_res["total"].values[0])
num_chargers = len(county_super)
num_evs = len(county_ev)

print("Chelan population (25–59):", population_25_59)
print("Existing chargers:", num_chargers)
print("Existing EVs:", num_evs)


In [ ]:

# ---------------- CHARGER TARGETS ----------------
area_per_charger = np.pi * RADIUS_MILES**2
lower_chargers = math.ceil(COUNTY_AREA_SQMI / area_per_charger)
upper_chargers = math.ceil(population_25_59 / POP_PER_CHARGER)

lower_chargers, upper_chargers


In [ ]:

# ---------------- LOWER / UPPER SCENARIOS ----------------
years = list(range(2025,2051))
rows = []

# LOWER
current = num_chargers
for y in years:
    current += max(0, math.ceil((lower_chargers-num_chargers)/len(years)))
    eff = BASE_2025_ADOPTION + (POLICY_TARGETS[2030]-BASE_2025_ADOPTION)*((y-2025)/5)*LOWER_P1_MOD
    rows.append(["Lower",y,current,eff])

# UPPER
current = num_chargers
for y in years:
    current += max(0, math.ceil((upper_chargers-num_chargers)/len(years)))
    eff = BASE_2025_ADOPTION + (POLICY_TARGETS[2030]-BASE_2025_ADOPTION)*((y-2025)/5)
    rows.append(["Upper",y,current,eff])

sc_df = pd.DataFrame(rows,columns=["Scenario","Year","Total_Chargers","Adoption_Rate"])
sc_df.head()


In [ ]:

# ---------------- MONOTONIC MONTE CARLO ----------------
yrs = sc_df["Year"].unique()
lo = sc_df.pivot(index="Year",columns="Scenario",values="Adoption_Rate")["Lower"].values
hi = sc_df.pivot(index="Year",columns="Scenario",values="Adoption_Rate")["Upper"].values

target = np.minimum((0.55*hi + 0.45*lo), TARGET_CAP)
target = np.maximum.accumulate(target)

paths = np.zeros((MC_RUNS,len(yrs)))
paths[:,0] = 0.35*lo[0] + 0.65*hi[0]

rng = np.random.default_rng(42)
for t in range(1,len(yrs)):
    noise = rng.normal(0,0.01,MC_RUNS)
    delta = np.maximum(0.3*(target[t]-paths[:,t-1]) + noise,0)
    paths[:,t] = np.maximum(paths[:,t-1]+delta,paths[:,t-1])

p50 = np.percentile(paths,50,axis=0)


In [ ]:

# ---------------- FINAL FORECAST ----------------
forecast = pd.DataFrame({
    "Year": yrs,
    "Adoption_P50": p50,
    "EVs_P50": (p50*population_25_59).astype(int)
})

forecast.tail()
